# Gathering Citation class information

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import grequests
import json
import asyncio
import concurrent.futures
import random
import asyncio
from aiohttp import ClientSession
import os
import aiohttp

ModuleNotFoundError: No module named 'grequests'

In [ ]:
citations = pd.read_csv('citations_patents_level.csv')
citations.head()

In [ ]:
citations.shape

In [ ]:
citations.groupby('patent_number').ngroups

For some reason or another, the version that filters non-design patents did not save. Let's fix that real quick

In [ ]:
def remove_non_design(df):
    return df[df.patent_number.str.contains('D')]
    

In [ ]:
citations = remove_non_design(citations)

In [ ]:
citations.groupby('patent_number').ngroups

In [ ]:
citations.to_csv('citations_patents_level.csv', index=False)

In [ ]:
citations.isnull().sum()

In [ ]:
citations.dropna(inplace=True)
citations.shape

Let's remove any duplicate patents that have been referenced by multiple patents

In [ ]:
citations.drop_duplicates(subset=['cited_patent_number'],inplace=True)
citations.shape

In [ ]:
941034/6777757

In [ ]:
needed_patents= citations['cited_patent_number'].sort_values()

In [ ]:
needed_patents.head()

generating all the URLs for the patents needed

In [26]:
urls = []
base_url = "http://www.patentsview.org/api/patents/query?"
field_list = "&f=[\"patent_number\",\"uspc_subclass_id\",\"uspc_mainclass_id\"]"
for i, value in needed_patents.iteritems():
    query = "q={{\"patent_number\":\"{}\"}}".format(value)
    urls.append(base_url + query + field_list)


In [ ]:
step = 1000
sucessful_responses = []
partition = 1000

In [ ]:
def pull_urls(urls):
    runloop(urls[i:i+step])

async def fetch(url, session):
    async with session.get(url) as response:
        print(response.status)
        if(response.status == 503):
            print('dead')
        else:
            sucessful_responses.append(await response.json())
        

async def bound_fetch(sem, url, session):
    # Getter function with semaphore.
    async with sem:
        await fetch(url, session)


async def run(r, urls):
    tasks = []
    # create instance of Semaphore
    sem = asyncio.Semaphore(1000)

    # Create client session that will ensure we dont open new connection
    # per each request.
    async with ClientSession() as session:
        for i in range(r):
            # pass Semaphore and session to every GET request
            task = asyncio.ensure_future(bound_fetch(sem, urls[i][0], session))
            tasks.append(task)

        responses = asyncio.gather(*tasks)
        await responses

def runloop(urls):
    number = step
    loop = asyncio.get_event_loop()

    future = asyncio.ensure_future(run(number, urls))
    loop.run_until_complete(future)

In [37]:
pd.DataFrame(urls).to_csv('citation_urls.csv', index=False, header=True)